In [ ]:
#THis is intended to capture audio from microphone
#This works but would like to reduce time to record and also potentially change voice

In [ ]:
# Install necessary libraries
!apt-get install -y portaudio19-dev libportaudio2 libportaudiocpp0
!pip install torch transformers speechrecognition pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (191 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123633 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Selecting previously un

In [ ]:
#I think this is unnecessary.  Delete if possible
#!pip install librosa soundfile

In [ ]:
!pip install pydub
!pip install ffmpeg

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=ff5c3b22fecd6cf1f4715020cf542bffa5823d68c57136cbdd29f84ca114cfbf
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:

# Import required libraries
import torch
import torchaudio
from transformers import AutoProcessor, SeamlessM4Tv2Model
import speech_recognition as sr
import numpy as np

from IPython.display import display, Javascript, Audio
from google.colab import output
import base64
import IPython
import wave
import time
import os

In [ ]:
from google.colab import output
from base64 import b64decode
import numpy as np
import io
import soundfile as sf
import librosa

from pydub import AudioSegment

In [ ]:
tick = time.time()

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
tick = time.time()
# Load processor and model
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2Model.from_pretrained(
    "facebook/seamless-m4t-v2-large",
    torch_dtype="float16"
).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/211k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/9.91M [00:00<?, ?B/s]

In [ ]:

# Function to record audio from the microphone
#recognizer = sr.Recognizer()

In [ ]:
# Function to record audio from the microphone with optimized parameters
def record_audio():
    # Record audio using the browser's microphone with optimized silence detection
    audio_base64 = output.eval_js('''
        async function recordAudio() {
            const constraints = { audio: true, echoCancellation: true };
            const stream = await navigator.mediaDevices.getUserMedia(constraints);
            const mediaRecorder = new MediaRecorder(stream);
            const data = [];
            mediaRecorder.ondataavailable = event => data.push(event.data);
            mediaRecorder.start();

            const audioCtx = new (window.AudioContext || window.webkitAudioContext)();
            const source = audioCtx.createMediaStreamSource(stream);
            const analyser = audioCtx.createAnalyser();
            source.connect(analyser);

            analyser.fftSize = 2048;
            const bufferLength = analyser.fftSize;
            const dataArray = new Uint8Array(bufferLength);

            let silenceTimer = 0;
            const silenceThreshold = 20;
            const silenceDuration = 3000; // Adjusted silence duration
            const minRecordDuration = 5000; // Adjusted minimum recording duration
            const startTime = Date.now();

            function checkSilence() {
                analyser.getByteTimeDomainData(dataArray);
                let sum = 0;
                for (let i = 0; i < bufferLength; i++) {
                    sum += (dataArray[i] - 128) * (dataArray[i] - 128);
                }
                const rms = Math.sqrt(sum / bufferLength);
                if (rms < silenceThreshold) {
                    silenceTimer += 100;
                    if (silenceTimer >= silenceDuration && Date.now() - startTime >= minRecordDuration) {
                        mediaRecorder.stop();
                    }
                } else {
                    silenceTimer = 0;
                }
                setTimeout(checkSilence, 100);
            }

            checkSilence();

            return new Promise((resolve) => {
                mediaRecorder.onstop = () => {
                    const audioBlob = new Blob(data, { type: 'audio/wav' });
                    const reader = new FileReader();
                    reader.onloadend = () => {
                        resolve(reader.result.split(',')[1]); // Return base64 data without the prefix
                    };
                    reader.readAsDataURL(audioBlob);
                };
            });
        }
        recordAudio();
    ''')

    # Decode the base64 string to bytes
    audio_bytes = b64decode(audio_base64)

    # Write the bytes to a file
    with open("recorded_audio.wav", "wb") as f:
        f.write(audio_bytes)
    print("Audio recorded successfully.")

    return audio_bytes

# Optimized resample_audio function using librosa
def resample_audio(audio_data, target_sample_rate=16000):
    # Load the audio using librosa
    waveform, sample_rate = librosa.load(audio_data, sr=None)

    # Resample to the target sample rate
    if sample_rate != target_sample_rate:
        waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr=target_sample_rate)

    # Normalize the waveform
    waveform = waveform / np.max(np.abs(waveform))

    # Ensure single-channel audio
    if waveform.ndim > 1:
        waveform = np.mean(waveform, axis=1)

    # Add batch dimension and move to GPU
    waveform = torch.from_numpy(waveform).unsqueeze(0).to(device)

    # Process the waveform using the processor
    inputs = processor(
        audios=waveform.squeeze().cpu().numpy(),
        sampling_rate=target_sample_rate,
        return_tensors="pt"
    )

    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move to GPU

    return inputs

In [ ]:
"""

def record_audio():
    # Record audio using the browser's microphone with automatic pause detection
    tick = time.time()
    audio_base64 = output.eval_js('''
        async function recordAudio() {
            const constraints = { audio: true, echoCancellation: true };
            const stream = await navigator.mediaDevices.getUserMedia(constraints);
            const mediaRecorder = new MediaRecorder(stream);
            const data = [];
            mediaRecorder.ondataavailable = event => data.push(event.data);
            mediaRecorder.start();

            const audioCtx = new (window.AudioContext || window.webkitAudioContext)();
            const source = audioCtx.createMediaStreamSource(stream);
            const analyser = audioCtx.createAnalyser();
            source.connect(analyser);

            analyser.fftSize = 2048;
            const bufferLength = analyser.fftSize;
            const dataArray = new Uint8Array(bufferLength);

            let silenceTimer = 0;
            const silenceThreshold = 20;
            const silenceDuration = 5000; // milliseconds
            const minRecordDuration = 10000; // minimum recording duration in milliseconds
            const startTime = Date.now();

            function checkSilence() {
                analyser.getByteTimeDomainData(dataArray);
                let sum = 0;
                for (let i = 0; i < bufferLength; i++) {
                    sum += (dataArray[i] - 128) * (dataArray[i] - 128);
                }
                const rms = Math.sqrt(sum / bufferLength);
                if (rms < silenceThreshold) {
                    silenceTimer += 100;
                    if (silenceTimer >= silenceDuration && Date.now() - startTime >= minRecordDuration) {
                        mediaRecorder.stop();
                    }
                } else {
                    silenceTimer = 0;
                }
                setTimeout(checkSilence, 100);
            }

            checkSilence();

            return new Promise((resolve) => {
                mediaRecorder.onstop = () => {
                    const audioBlob = new Blob(data, { type: 'audio/wav' });
                    const reader = new FileReader();
                    reader.onloadend = () => {
                        resolve(reader.result.split(',')[1]); // Return base64 data without the prefix
                    };
                    reader.readAsDataURL(audioBlob);
                };
            });
        }
        recordAudio();
    ''')


    # Decode the base64 string to bytes
    audio_bytes = b64decode(audio_base64)

    # Remove the file if it exists
    if os.path.exists("recorded_audio.wav"):
        os.remove("recorded_audio.wav")
        print("Previous audio file removed.")

    # Write the bytes to a file
    with open("recorded_audio.wav", "wb") as f:
        f.write(audio_bytes)
        print("Audio file saved as 'recorded_audio.wav'.")
    print("Audio recorded successfully.")
    tock = time.time()
    print("Record time: ", str(round(tock - tick, 2)), " seconds")

    return audio_bytes
    """

'\n\ndef record_audio():\n    # Record audio using the browser\'s microphone with automatic pause detection\n    tick = time.time()\n    audio_base64 = output.eval_js(\'\'\'\n        async function recordAudio() {\n            const constraints = { audio: true, echoCancellation: true };\n            const stream = await navigator.mediaDevices.getUserMedia(constraints);\n            const mediaRecorder = new MediaRecorder(stream);\n            const data = [];\n            mediaRecorder.ondataavailable = event => data.push(event.data);\n            mediaRecorder.start();\n\n            const audioCtx = new (window.AudioContext || window.webkitAudioContext)();\n            const source = audioCtx.createMediaStreamSource(stream);\n            const analyser = audioCtx.createAnalyser();\n            source.connect(analyser);\n\n            analyser.fftSize = 2048;\n            const bufferLength = analyser.fftSize;\n            const dataArray = new Uint8Array(bufferLength);\n\n           

In [ ]:
"""
def resample_audio(audio_data, target_sample_rate):

  # Load the downloaded WAV file into tensor and resample
  tick = time.time()

  # Load the audio using torchaudio
  waveform, sample_rate = torchaudio.load(audio_data)

  # Resample to 16 kHz if necessary
  #target_sample_rate = 16000
  if sample_rate != target_sample_rate:
      resampler = torchaudio.transforms.Resample(sample_rate, target_sample_rate)
      waveform = resampler(waveform)

  # Normalize the waveform
  waveform = waveform / waveform.abs().max()

  # Ensure single-channel audio (if necessary)
  if waveform.size(0) > 1:
      waveform = waveform.mean(dim=0, keepdim=True)

  # Add batch dimension
  waveform = waveform.unsqueeze(0).to(device)  # Move to GPU

  # Process the waveform using the processor
  inputs = processor(
      audios=waveform.squeeze().cpu().numpy(),  # Move to CPU and convert to NumPy
      sampling_rate=target_sample_rate,
      return_tensors="pt"
  )

  inputs = {key: value.to(device) for key, value in inputs.items()}  # Move to GPU


  tock = time.time()
  print("Load time: ", str(round(tock - tick, 2)), " seconds")

  return inputs
  """

'\ndef resample_audio(audio_data, target_sample_rate):\n\n  # Load the downloaded WAV file into tensor and resample\n  tick = time.time()\n\n  # Load the audio using torchaudio\n  waveform, sample_rate = torchaudio.load(audio_data)\n\n  # Resample to 16 kHz if necessary\n  #target_sample_rate = 16000\n  if sample_rate != target_sample_rate:\n      resampler = torchaudio.transforms.Resample(sample_rate, target_sample_rate)\n      waveform = resampler(waveform)\n\n  # Normalize the waveform\n  waveform = waveform / waveform.abs().max()\n\n  # Ensure single-channel audio (if necessary)\n  if waveform.size(0) > 1:\n      waveform = waveform.mean(dim=0, keepdim=True)\n\n  # Add batch dimension\n  waveform = waveform.unsqueeze(0).to(device)  # Move to GPU\n\n  # Process the waveform using the processor\n  inputs = processor(\n      audios=waveform.squeeze().cpu().numpy(),  # Move to CPU and convert to NumPy\n      sampling_rate=target_sample_rate,\n      return_tensors="pt"\n  )\n\n  inputs 

In [ ]:
def translate_audio(inputs):
    # Generate translation
    tick = time.time()
    outputs = model.generate(**inputs, tgt_lang="eng")[0]
    translation = outputs.cpu().numpy().squeeze()  # Move to CPU for playback
    tock = time.time()
    print("Translation time: ", str(round(tock - tick, 2)), " seconds")
    return translation

In [ ]:
print("Please speak into the microphone...")
audio_bytes = record_audio()
input_tensors=resample_audio(audio_bytes, target_sample_rate=16000)
translation = translate_audio(input_tensors)
Audio(translation, rate=16000)

Please speak into the microphone...
Audio recorded successfully.


LibsndfileError: Error opening b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x15I\xa9f\x99*\xd7\xb1\x83\x0fB@M\x80\x86ChromeWA\x86Chrome\x16T\xaek\xbf\xae\xbd\xd7\x81\x01s\xc5\x87/\xc8\x1f\t>\xb1\x04\x83\x81\x02\x86\x86A_OPUSc\xa2\x93OpusHead\x01\x01\x00\x00\x80\xbb\x00\x00\x00\x00\x00\xe1\x8d\xb5\x84G;\x80\x00\x9f\x81\x01bd\x81 \x1fC\xb6u\x01\xff\xff\xff\xff\xff\xff\xff\xe7\x81\x00\xa3D\xcf\x81\x00\x00\x80\xfb\x83\xfdC\xfe\x15\x7f2\xcd@\xe3C\x02\x1bf\xd9\xc9\xe1\x9c3\x12\xc3J\\\xb9\x0f\xffzY@\xbc\xd4\x97/*\x08I\xca\xc9\xedI\\\x9b\x12\xcc\x15@\x1a59n\xbciSw\x198\xd7\xa9\xcaM\x87O\xe9\xf4\xe6\xa0\xe7\xf4\x8b\xa1VY\xa4gf\xbb0\xb1o\x0f\x08\xb9\xa1\x13\xce\xec\xae\xb1w\x16\xa1\xfdF\xee\xb8\xfeB\x8b\xa6\x89{X\x08n\xaek\xf1\xad\x00\xdan\x9bo\xa6\x9f\xbc\x00Mf\x8c\xd7\xa6\x15\xaf[\xba\x93\xc3\xd5;1\xe8\x9dY\x8c\xdb\x8f\x85)>iJ\x98v\r\xbaX\x0cg\x10\xc2m7O\x1f|\x1br\x85\xfe\xdd\x9f\x1cK\x1

In [ ]:
####TRY THIS WHEN YOU RETURN>. IT SHOULD DO A CONSTANT TRANSLATION

from IPython.display import display, Audio
from google.colab import output
import time
import uuid
import numpy as np
import wave

# Assuming these functions are defined
# def record_audio():
# def resample_audio(audio_bytes, target_sample_rate):
# def translate_audio(input_tensors):

def play_translation_and_wait(translation, filename='translation.wav'):
    # Save translation to WAV file
    with wave.open(filename, 'wb') as wav_file:
        wav_file.setnchannels(1)  # mono
        wav_file.setsampwidth(2)  # 16-bit
        wav_file.setframerate(16000)
        wav_file.writeframes((translation * 32767).astype(np.int16).tobytes())

    # Calculate the duration of the translation audio
    framerate = 16000
    duration = len(translation) / framerate  # in seconds

    # Display the audio player
    display(Audio(filename, autoplay=True))

    # Wait for the duration plus a small buffer
    time.sleep(duration + 1)

# Main loop
while True:
    try:
        # Record audio
        audio_bytes = record_audio()

        # Resample audio
        input_tensors = resample_audio(audio_bytes, target_sample_rate=16000)

        # Translate audio
        translation = translate_audio(input_tensors)

        # Play translation and wait for it to finish
        play_translation_and_wait(translation)

    except Exception as e:
        print(f"An error occurred: {e}")
        break  # Exit the loop on error

Previous audio file removed.
Audio file saved as 'recorded_audio.wav'.
Audio recorded successfully.
Record time:  14.54  seconds
Load time:  0.09  seconds
Translation time:  1.25  seconds


Previous audio file removed.
Audio file saved as 'recorded_audio.wav'.
Audio recorded successfully.
Record time:  12.64  seconds
Load time:  0.07  seconds
Translation time:  1.29  seconds


Previous audio file removed.
Audio file saved as 'recorded_audio.wav'.
Audio recorded successfully.
Record time:  15.41  seconds
Load time:  0.09  seconds
Translation time:  1.43  seconds


Previous audio file removed.
Audio file saved as 'recorded_audio.wav'.
Audio recorded successfully.
Record time:  10.59  seconds
Load time:  0.09  seconds
Translation time:  1.9  seconds


Previous audio file removed.
Audio file saved as 'recorded_audio.wav'.
Audio recorded successfully.
Record time:  10.62  seconds
Load time:  0.06  seconds
Translation time:  1.13  seconds


KeyboardInterrupt: 

In [ ]:
#THis is the original untranslated recording for testing purposes
from IPython.display import Audio, display
audio_to_play = Audio("recorded_audio.wav")
display(audio_to_play)

new audio?


torch version: 2.5.1+cu121
torchaudio version: 2.5.1+cu121
librosa version: 0.10.2.post1
transformers version: 4.46.3
Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Starting recording...
An error occurred: NotSupportedError: Failed to construct 'MediaRecorder': Failed to initialize native MediaRecorder the type provided (audio/wav) is not supported.
